# Random Forest

- Random Forest is combination of many decision trees
- It is a classification algorithm.

Why do we need Random Forest over Decision Trees?
- Though Decision Trees are easy to build, use and interpret, but they are inaccurate
- DTs are not very good with unseen data so our Model may not work as desired
- Random Forest = Simplicity of DT + Very Good Accuracy

## Input

1. .csv - produced by pre_processing.ipynb
2. The pre_processed input data includes following techniques:
    #TODO

## Output/Analysis

1. Visualising the accuracy of RF with k-fold validation.
2. Comparing the accurancy of RF model with and without PCA.   

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Performing k-fold Validation on the dataset

# Classification of input data with RF 

In [12]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)

In [13]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [14]:
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [15]:
print(clf.predict([[0, 0, 0, 0]]))

[1]


# Classification with RF after Dimension Reduction (using PCA)